In [3]:
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re

# XML file path
xml_file_path = f'D:\W\RELEX\RELEX\Future_List_Cost\FUTRLIST.IN07.20231206100039 (Oracle File)'
# Open the XML file
with open(xml_file_path, 'r', encoding='utf-8') as file:
    xml_data = file.read()
print(xml_data)
# Parse the XML data
root = ET.fromstring(xml_data)
print(root)

<LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00046.1800</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA4618</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000001<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
<LIST_COST_MAINT><BICEPSITEM>250494237</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00024.2600</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA2426</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000002<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                         

ParseError: mismatched tag: line 1, column 361 (<string>)

In [4]:
import xml.etree.ElementTree as ET

# Print the XML data
print(xml_data)

# Try to parse the XML data
try:
    root = ET.fromstring(xml_data)
    print(root)
except ET.ParseError as e:
    print(f"Error parsing XML: {e}")


<LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00046.1800</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA4618</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000001<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
<LIST_COST_MAINT><BICEPSITEM>250494237</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00024.2600</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA2426</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000002<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                         

In [5]:
file_path = f'D:\W\RELEX\RELEX\Future_List_Cost\FUTRLIST.IN07.20231206100039 (Oracle File)'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Iterate over each line and insert into the PostgreSQL table
for line in lines:
    # Extract values between angle brackets
    values = [value.strip() for value in line.split('<')[1:-1]]
values

['LIST_COST_MAINT>',
 'BICEPSITEM>251580418',
 '/BICEPSITEM>',
 'FIRSTORDERDATE>010524',
 '/FIRSTORDERDATE>',
 'LASTORDERDATE>020424',
 '/LASTORDERDATE>',
 'ITEMLISTFUTURE>00019.9200',
 '/ITEMLISTFUTURE>',
 'DATESELLEFF>020524',
 '/DATESELLEFF>',
 'DEALMANUFACTURENUMBER>010540204ORA1992',
 '/DEALMANUFACTURENUMBER>',
 'LISTCOSTADD>',
 '/LISTCOSTADD>',
 'TRANSACTIONID>L120623000000000000008',
 'TRANSACTIONID>']

In [7]:
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re


# XML file path
xml_file_path = f'D:\W\RELEX\RELEX\Future_List_Cost\FUTRLIST.IN07.20231206100039 (Oracle File)'
# Open the XML file
with open(xml_file_path, 'r', encoding='utf-8') as file:
    xml_data = file.read()
# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Table name
table_name = "cspoms.XXPO_ORCL2CSPOMS_VEND_STG".upper()

# Split the XML data into lines
lines = xml_data.splitlines()

# Regular expression pattern to match the elements
pattern = re.compile(r"<([^>]+)>([^<]+)</\1>")

# print('tag_to_column : ',len(tag_to_column))
# Loop through each line in the XML data
for line in lines:
    # Skip empty lines or lines containing only whitespace
    if not line.strip():
        continue

    # Define a dictionary to store values
    element_values = {}

    # Find all matches in the text data
    matches = re.findall(pattern, line)

    # Store the matches in the dictionary
    for tag, value in matches:
        element_values[tag] = value.strip()
   
    print('element_values:############',element_values)
    # Now 'BIC_VENDOR_FACILITY' and updated 'BICEPSVENDOR' are present in element_values
    # Replace column names with the ones from tag_to_column
    mapped_values = {tag_to_column[key]: value for key, value in element_values.items() if key in tag_to_column}
    print('mapped_values:########',mapped_values)
    # Generate SQL INSERT statement
    columns_str = ",".join(mapped_values.keys())
    values_str = ",".join(f"'{v}'" for v in mapped_values.values())

    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str});"
    print(insert_query)



element_values:############ {'BICEPSITEM': '250473199', 'FIRSTORDERDATE': '120723', 'LASTORDERDATE': '010624', 'ITEMLISTFUTURE': '00046.1800', 'DATESELLEFF': '010724', 'DEALMANUFACTURENUMBER': '120730106ORA4618'}


NameError: name 'tag_to_column' is not defined

In [14]:
import psycopg2

file_path = f'D:\W\RELEX\RELEX\Future_List_Cost\FUTRLIST.IN07.20231206100039 (Oracle File)'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Create a cursor
cursor = conn.cursor()
print('conn establisted')
# Function to extract values from a multiline element
def extract_values(element_text):
    values = {}
    for line in element_text.splitlines():
        if '<' in line and '>' in line:
            tag_start = line.index('<') + 1
            tag_end = line.index('>')
            tag = line[tag_start:tag_end]
            value_start = tag_end + 1
            value = line[value_start:].strip()
            values[tag] = value
    return values

# Iterate over each line and insert into the PostgreSQL table
current_element_text = ""
for line in lines:
    if "<LIST_COST_MAINT>" in line:
        current_element_text = line
        print(line)
    elif "<LIST_COST_MAINT>" in line:
        current_element_text += line
        element_values = extract_values(current_element_text)
        print(element_values)
        # Example SQL insert query (replace "your_table" and column names with your actual table and columns)
        sql_insert = f"""
        INSERT INTO your_table (bicepsitem, firstorderdate, lastorderdate, itemlistfuture, dateselleff, dealmanufacturenumber, listcostadd, transactionid)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """
        print(sql_insert)

        # Execute the insert query
        cursor.execute(sql_insert, (
            element_values.get('BICEPSITEM', ''),
            element_values.get('FIRSTORDERDATE', ''),
            element_values.get('LASTORDERDATE', ''),
            element_values.get('ITEMLISTFUTURE', ''),
            element_values.get('DATESELLEFF', ''),
            element_values.get('DEALMANUFACTURENUMBER', ''),
            element_values.get('LISTCOSTADD', ''),
            element_values.get('TRANSACTIONID', '')
        ))
        print(current_element_text)
        current_element_text = ""
    else:
        current_element_text += line
        print(current_element_text)
print('end of program')
# Commit the changes and close the connection
# conn.commit()
# cursor.close()
# conn.close()


conn establisted
<LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00046.1800</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA4618</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000001<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            

<LIST_COST_MAINT><BICEPSITEM>250494237</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00024.2600</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA2426</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000002<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                       

In [15]:
file_path = f'D:\W\RELEX\RELEX\Future_List_Cost\FUTRLIST.IN07.20231206100039 (Oracle File)'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Function to extract values from a multiline element
def extract_values(element_text):
    values = {}
    lines = element_text.splitlines()
    for line in lines:
        while '<' in line and '>' in line:
            tag_start = line.index('<') + 1
            tag_end = line.index('>')
            tag = line[tag_start:tag_end]
            value_start = tag_end + 1
            value = line[value_start:].strip()
            values[tag] = value
            line = line[tag_end + 1:]
    return values

# Iterate over each line and insert into the PostgreSQL table
current_element_text = ""
for line in lines:
    current_element_text += line
    if "</LIST_COST_MAINT>" in line:
        element_values = extract_values(current_element_text)
print(element_values)



{'LIST_COST_MAINT': '<BICEPSITEM>251580418</BICEPSITEM> <FIRSTORDERDATE>010524</FIRSTORDERDATE> <LASTORDERDATE>020424</LASTORDERDATE> <ITEMLISTFUTURE>00019.9200</ITEMLISTFUTURE> <DATESELLEFF>020524</DATESELLEFF> <DEALMANUFACTURENUMBER>010540204ORA1992</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000008<TRANSACTIONID> </LIST_COST_MAINT>', 'BICEPSITEM': '251580418</BICEPSITEM> <FIRSTORDERDATE>010524</FIRSTORDERDATE> <LASTORDERDATE>020424</LASTORDERDATE> <ITEMLISTFUTURE>00019.9200</ITEMLISTFUTURE> <DATESELLEFF>020524</DATESELLEFF> <DEALMANUFACTURENUMBER>010540204ORA1992</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000008<TRANSACTIONID> </LIST_COST_MAINT>', '/BICEPSITEM': '<FIRSTORDERDATE>010524</FIRSTORDERDATE> <LASTORDERDATE>020424</LASTORDERDATE> <ITEMLISTFUTURE>00019.9200</ITEMLISTFUTURE> <DATESELLEFF>020524</DATESELLEFF> <DEALMANUFACTURENUMBER>010540204ORA1992</DEALMANUFACTURENUMBER> <LISTCOSTADD><

In [21]:
file_path = f'D:\W\RELEX\RELEX\Future_List_Cost\FUTRLIST.IN07.20231206100039 (Oracle File)'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
lines

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\W\\RELEX\\RELEX\\Future_List_Cost\\FUTRLIST.IN07.20231206100039 (Oracle File)'

In [18]:
from lxml import etree

schema = etree.XMLSchema(etree.parse('D:\W\RELEX\RELEX\Future_List_Cost\Inbound_files\Future.xsd'))
parser = etree.XMLParser(schema=schema)

try:
    tree = etree.parse('D:\W\RELEX\RELEX\Future_List_Cost\Inbound_files\FUTRLIST.IN07.20231206100039 (Oracle File)', parser)
except etree.XMLSyntaxError as e:
    print(f"XML Validation Error: {e}")


XML Validation Error: Element 'TRANSACTIONID': Element content is not allowed, because the content type is a simple type definition. (<string>, line 0)


In [20]:
import xml.etree.ElementTree as ET

def parse_list_cost_maint(element):
    data = {}
    for child in element:
        data[child.tag] = child.text
    return data

xml_data = """  <LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00046.1800</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA4618</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000001<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
<LIST_COST_MAINT><BICEPSITEM>250494237</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00024.2600</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA2426</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000002<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
<LIST_COST_MAINT><BICEPSITEM>250091739</BICEPSITEM> <FIRSTORDERDATE>030824</FIRSTORDERDATE> <LASTORDERDATE>040724</LASTORDERDATE> <ITEMLISTFUTURE>00014.2000</ITEMLISTFUTURE> <DATESELLEFF>040824</DATESELLEFF> <DEALMANUFACTURENUMBER>030840407ORA1420</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000003<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
<LIST_COST_MAINT><BICEPSITEM>250952159</BICEPSITEM> <FIRSTORDERDATE>030824</FIRSTORDERDATE> <LASTORDERDATE>040724</LASTORDERDATE> <ITEMLISTFUTURE>00013.5200</ITEMLISTFUTURE> <DATESELLEFF>040824</DATESELLEFF> <DEALMANUFACTURENUMBER>030840407ORA1352</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000004<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
<LIST_COST_MAINT><BICEPSITEM>250494237</BICEPSITEM> <FIRSTORDERDATE>030824</FIRSTORDERDATE> <LASTORDERDATE>040724</LASTORDERDATE> <ITEMLISTFUTURE>00022.6500</ITEMLISTFUTURE> <DATESELLEFF>040824</DATESELLEFF> <DEALMANUFACTURENUMBER>030840407ORA2265</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000005<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
<LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIRSTORDERDATE>030824</FIRSTORDERDATE> <LASTORDERDATE>040724</LASTORDERDATE> <ITEMLISTFUTURE>00040.9300</ITEMLISTFUTURE> <DATESELLEFF>040824</DATESELLEFF> <DEALMANUFACTURENUMBER>030840407ORA4093</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000006<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
<LIST_COST_MAINT><BICEPSITEM>251582038</BICEPSITEM> <FIRSTORDERDATE>010524</FIRSTORDERDATE> <LASTORDERDATE>020424</LASTORDERDATE> <ITEMLISTFUTURE>00187.2000</ITEMLISTFUTURE> <DATESELLEFF>020524</DATESELLEFF> <DEALMANUFACTURENUMBER>010540204ORA1820</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000007<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
<LIST_COST_MAINT><BICEPSITEM>251580418</BICEPSITEM> <FIRSTORDERDATE>010524</FIRSTORDERDATE> <LASTORDERDATE>020424</LASTORDERDATE> <ITEMLISTFUTURE>00019.9200</ITEMLISTFUTURE> <DATESELLEFF>020524</DATESELLEFF> <DEALMANUFACTURENUMBER>010540204ORA1992</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000008<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            

"""

root = ET.fromstring(xml_data)

for list_cost_maint_element in root.findall('LIST_COST_MAINT'):
    data = parse_list_cost_maint(list_cost_maint_element)
    print(data)


ParseError: mismatched tag: line 1, column 363 (<string>)

In [23]:
import os
import xml.etree.ElementTree as ET
import psycopg2

# Define the PostgreSQL connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Define the root directory path
root_directory = r'D:\W\RELEX\RELEX\Future_List_Cost\Inbound_files'

# Specify the file name
# file_name = 'future.xml'
file_name = f'FUTRLIST.IN07.20231206100039 (Oracle File)'

# Join the root directory path and the file name
file_path = os.path.join(root_directory, file_name)

# Open the file and parse each line
with open(file_path, 'r', encoding='utf-8') as file:
    # Establish a database connection and execute the INSERT statement
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cursor:
            for line in file:
                try:
                    # Parse each line as XML
                    root = ET.fromstring(line.strip())

                    # Access the elements within each <LIST_COST_MAINT>
                    BICEPSITEM = root.find('BICEPSITEM').text
                    FIRSTORDERDATE = root.find('FIRSTORDERDATE').text
                    LASTORDERDATE = root.find('LASTORDERDATE').text
                    ITEMLISTFUTURE = root.find('ITEMLISTFUTURE').text
                    DATESELLEFF = root.find('DATESELLEFF').text
                    DEALMANUFACTURENUMBER = root.find('DEALMANUFACTURENUMBER').text
                    LISTCOSTADD = root.find('LISTCOSTADD').text
                    TRANSACTIONID = root.find('TRANSACTIONID').text

                    # Execute the INSERT statement for each line
                    cursor.execute("""
                        INSERT INTO LIST_COST_MAINT_STG (
                            BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                            ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                            LISTCOSTADD, TRANSACTIONID
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    """, (
                        BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                        ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                        LISTCOSTADD, TRANSACTIONID
                    ))

                except ET.ParseError as e:
                    print(f"Error parsing line: {line.strip()}")
                    print(f"ParseError: {e}")

# Commit the changes to the database
conn.commit()


Error parsing line: <LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00046.1800</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA4618</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000001<TRANSACTIONID> </LIST_COST_MAINT>
ParseError: mismatched tag: line 1, column 361
Error parsing line: <LIST_COST_MAINT><BICEPSITEM>250494237</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00024.2600</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA2426</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000002<TRANSACTIONID> </LIST_COST_MAINT>
ParseError: mismatched tag: line 1, column 361
Error parsing line: <LIST_COST_MAINT><BICEPSITEM>250091739</BICEPSITEM> <FIRSTORDERDATE>030824</FIRSTORDERDATE